In [21]:
import pandas as pd
import numpy as np
import os

In [22]:
dates = pd.date_range(start='1/1/2023', end='5/31/2024', freq='YS')

In [23]:
all_dfs = []

for date in dates:
    season_start = date.year
    season_end = date.year + 1
    season = f'{season_start}-{season_end}'
    print(season)
    
    df = pd.read_html(f'https://fbref.com/en/comps/9/{season}/schedule/{season}-Premier-League-Scores-and-Fixtures', attrs={"id": f"sched_{season}_9_1"})[0]
    df['Season'] = season
    all_dfs.append(df)

2023-2024
2024-2025


In [24]:
# put all the seasons together and drop the week wiht no games
df = pd.concat(all_dfs)
df = df.dropna(subset=['Wk'])
df['Wk'] = df['Wk'].astype(int)
df.drop(columns=['Match Report', 'Notes','Attendance'], inplace=True)

In [25]:
today = pd.to_datetime('today').date()
# convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date

# create a new df filter out the rows where the date is greater than the current day
df = df[df['Date'] < today]

In [26]:
df[['home_goals', 'away_goals']] = df['Score'].str.split('–', expand=True).astype(int)

# rename xG and xG.1 columns to home_xG and away_xG
df.rename(columns={'xG': 'home_xg', 'xG.1': 'away_xg'}, inplace=True)

In [27]:
def get_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'Home Win'
    elif row['home_goals'] < row['away_goals']:
        return 'Away Win'
    else:
        return 'Draw'
    
df['Result'] = df.apply(get_result, axis=1)

In [28]:
df = pd.get_dummies(df, columns=['Day'])
df.reset_index(drop=True, inplace=True)

In [29]:
for x in df.Home.unique():
    temp_df = df[(df['Home'] == x) | (df['Away'] == x)]
    temp_df = temp_df.sort_values(['Date'])

    temp_df['goal_value_to_calculate'] = temp_df.apply(lambda y: y['home_goals'] if y['Home'] == x else y['away_goals'], axis=1)
    temp_df['rolling_avg_goals'] = temp_df['goal_value_to_calculate'].rolling(window = 5, closed = 'left', min_periods = 1).mean()
    temp_df['xg_value_to_calculate'] = temp_df.apply(lambda y: y['home_xg'] if y['Home'] == x else y['away_xg'], axis=1)
    temp_df['rolling_avg_xG'] = temp_df['xg_value_to_calculate'].rolling(window = 5, closed = 'left', min_periods = 1).mean()

    for index, row in temp_df.iterrows():
        if row['Home'] == x:
            df.at[index, 'home_rolling_avg_goals'] = row['rolling_avg_goals']
            df.at[index, 'home_rolling_avg_xG'] = row['rolling_avg_xG']
        else:
            df.at[index, 'away_rolling_avg_goals'] = row['rolling_avg_goals']
            df.at[index, 'away_rolling_avg_xG'] = row['rolling_avg_xG']

In [30]:
df = df.dropna(subset=['home_rolling_avg_goals', 'away_rolling_avg_goals', 'home_rolling_avg_xG', 'away_rolling_avg_xG'])

In [31]:
# create a new column for the central moving average of the home goals
df['home_goals_cma'] = df['home_goals'].expanding().mean()
df['home_xg_cma'] = df['home_xg'].expanding().mean()

# away goals
df['away_goals_cma'] = df['away_goals'].expanding().mean()
df['away_xg_cma'] = df['away_xg'].expanding().mean()

# create a new column by dividing the home goals by central moving average
df['home_goals_div_cma'] = df['home_goals'] / df['home_goals_cma']
df['home_xg_div_cma'] = df['home_xg'] / df['home_xg_cma']

# away goals
df['away_goals_div_cma'] = df['away_goals'] / df['away_goals_cma']
df['away_xg_div_cma'] = df['away_xg'] / df['away_xg_cma']

In [32]:
# create a new column called quarter. divide the season into 4 quarters using the Wk column and assign a value between 1 and 4
df['quarter'] = np.where(df['Wk'].astype(int) <= 9, 1,
                np.where(df['Wk'].astype(int) <= 18, 2,
                np.where(df['Wk'].astype(int) <= 27, 3, 4)))

# using home_goals_div_cma, take the mean of the column for each quarter. put into new column called home_goals_seasonality
df['home_goals_seasonality'] = df.groupby('quarter')['home_goals_div_cma'].transform('mean')
df['home_xg_seasonality'] = df.groupby('quarter')['home_xg_div_cma'].transform('mean')

# away goals
df['away_goals_seasonality'] = df.groupby('quarter')['away_goals_div_cma'].transform('mean')
df['away_xg_seasonality'] = df.groupby('quarter')['away_xg_div_cma'].transform('mean')

# create a new column called home_goals_deseasonalised by dividing home_goals by home_goals_seasonality
df['home_goals_deseasonalised'] = df['home_goals'] / df['home_goals_seasonality']
df['home_xg_deseasonalised'] = df['home_xg'] / df['home_xg_seasonality']

# away goals
df['away_goals_deseasonalised'] = df['away_goals'] / df['away_goals_seasonality']
df['away_xg_deseasonalised'] = df['away_xg'] / df['away_xg_seasonality']

In [33]:
df[df['Season'] == '2024-2025']

,Wk,Date,Time,Home,home_xg,Score,away_xg,Away,Venue,Referee,...,away_xg_div_cma,quarter,home_goals_seasonality,home_xg_seasonality,away_goals_seasonality,away_xg_seasonality,home_goals_deseasonalised,home_xg_deseasonalised,away_goals_deseasonalised,away_xg_deseasonalised
380,1,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,Old Trafford,Robert Jones,...,0.297633,1,0.925219,0.963086,0.949344,0.932692,1.080825,2.491988,0.000000,0.428866
383,1,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,The City Ground,Michael Oliver,...,0.893157,1,0.925219,0.963086,0.949344,0.932692,1.080825,1.349827,1.053359,1.286598
384,1,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,Goodison Park,Simon Hooper,...,1.041899,1,0.925219,0.963086,0.949344,0.932692,0.000000,0.519164,3.160078,1.501031
385,1,2024-08-17,15:00,Arsenal,1.2,2–0,0.5,Wolves,Emirates Stadium,Jarred Gillett,...,0.372733,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.245994,0.000000,0.536083
386,1,2024-08-17,17:30,West Ham,2.3,1–2,2.0,Aston Villa,London Stadium,Tony Harrington,...,1.488982,1,0.925219,0.963086,0.949344,0.932692,1.080825,2.388155,2.106718,2.144330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,8,2024-10-19,17:30,Bournemouth,1.8,2–0,0.7,Arsenal,Vitality Stadium,Robert Jones,...,0.525089,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.868991,0.000000,0.750516
457,8,2024-10-20,14:00,Wolves,0.8,1–2,1.6,Manchester City,Molineux Stadium,Chris Kavanagh,...,1.199663,1,0.925219,0.963086,0.949344,0.932692,1.080825,0.830663,2.106718,1.715464
458,8,2024-10-20,16:30,Liverpool,1.9,2–1,1.0,Chelsea,Anfield,John Brooks,...,0.750210,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.972824,1.053359,1.072165
459,8,2024-10-21,20:00,Nott'ham Forest,1.7,1–0,1.0,Crystal Palace,The City Ground,Tim Robinson,...,0.750630,1,0.925219,0.963086,0.949344,0.932692,1.080825,1.765158,0.000000,1.072165


In [34]:
display(df)

,Wk,Date,Time,Home,home_xg,Score,away_xg,Away,Venue,Referee,...,away_xg_div_cma,quarter,home_goals_seasonality,home_xg_seasonality,away_goals_seasonality,away_xg_seasonality,home_goals_deseasonalised,home_xg_deseasonalised,away_goals_deseasonalised,away_xg_deseasonalised
10,2,2023-08-18,19:45,Nott'ham Forest,1.4,2–1,0.5,Sheffield Utd,The City Ground,Peter Bankes,...,1.000000,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.453660,1.053359,0.536083
11,2,2023-08-19,15:00,Fulham,0.5,0–3,3.7,Brentford,Craven Cottage,Darren Bond,...,1.761905,1,0.925219,0.963086,0.949344,0.932692,0.000000,0.519164,3.160078,3.967011
12,2,2023-08-19,15:00,Liverpool,3.0,3–1,1.3,Bournemouth,Anfield,Thomas Bramall,...,0.709091,1,0.925219,0.963086,0.949344,0.932692,3.242476,3.114985,1.053359,1.393815
13,2,2023-08-19,15:00,Wolves,2.1,1–4,2.2,Brighton,Molineux Stadium,Andy Madley,...,1.142857,1,0.925219,0.963086,0.949344,0.932692,1.080825,2.180490,4.213437,2.358763
14,2,2023-08-19,17:30,Tottenham,1.7,2–0,2.1,Manchester Utd,Tottenham Hotspur Stadium,Michael Oliver,...,1.071429,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.765158,0.000000,2.251547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,8,2024-10-19,17:30,Bournemouth,1.8,2–0,0.7,Arsenal,Vitality Stadium,Robert Jones,...,0.525089,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.868991,0.000000,0.750516
457,8,2024-10-20,14:00,Wolves,0.8,1–2,1.6,Manchester City,Molineux Stadium,Chris Kavanagh,...,1.199663,1,0.925219,0.963086,0.949344,0.932692,1.080825,0.830663,2.106718,1.715464
458,8,2024-10-20,16:30,Liverpool,1.9,2–1,1.0,Chelsea,Anfield,John Brooks,...,0.750210,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.972824,1.053359,1.072165
459,8,2024-10-21,20:00,Nott'ham Forest,1.7,1–0,1.0,Crystal Palace,The City Ground,Tim Robinson,...,0.750630,1,0.925219,0.963086,0.949344,0.932692,1.080825,1.765158,0.000000,1.072165


In [35]:
df = df.dropna(subset=['home_rolling_avg_goals', 'away_rolling_avg_goals', 'home_rolling_avg_xG', 'away_rolling_avg_xG'])
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

df['Season'] = df['Season'].apply(lambda x: x.split('-')[1])

In [36]:
df['Season'] = df['Season'].astype(int)

In [37]:
# df.to_csv('previous_matches_clean.csv')

In [40]:
df[df['Season'] == 2025]['Home'].value_counts()

Home
Manchester Utd     4
Nott'ham Forest    4
Leicester City     4
Liverpool          4
Wolves             4
Bournemouth        4
Aston Villa        4
Southampton        4
Manchester City    4
Crystal Palace     4
Fulham             4
Tottenham          4
Brighton           4
Chelsea            4
Brentford          4
West Ham           4
Arsenal            4
Everton            4
Ipswich Town       3
Newcastle Utd      3
Name: count, dtype: int64

In [39]:
df[(df['Season'] == 2025) & (df['Home'] == 'Manchester Utd')]

,Wk,Date,Time,Home,home_xg,Score,away_xg,Away,Venue,Referee,...,quarter,home_goals_seasonality,home_xg_seasonality,away_goals_seasonality,away_xg_seasonality,home_goals_deseasonalised,home_xg_deseasonalised,away_goals_deseasonalised,away_xg_deseasonalised,Day
380,1,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,Old Trafford,Robert Jones,...,1,0.925219,0.963086,0.949344,0.932692,1.080825,2.491988,0.000000,0.428866,Friday
409,3,2024-09-01,16:00,Manchester Utd,1.4,0–3,1.8,Liverpool,Old Trafford,Anthony Taylor,...,1,0.925219,0.963086,0.949344,0.932692,0.000000,1.453660,3.160078,1.929897,Sunday
438,6,2024-09-29,16:30,Manchester Utd,1.0,0–3,4.4,Tottenham,Old Trafford,Chris Kavanagh,...,1,0.925219,0.963086,0.949344,0.932692,0.000000,1.038328,3.160078,4.717527,Sunday
455,8,2024-10-19,15:00,Manchester Utd,1.3,2–1,0.9,Brentford,Old Trafford,Samuel Barrott,...,1,0.925219,0.963086,0.949344,0.932692,2.161651,1.349827,1.053359,0.964949,Saturday
